In [1]:
# Move working directory from notebooks/ to the repo root
%cd ..

c:\Users\lwhieldon\KaggleAgentsCapstone\PawConnect


In [2]:
import asyncio
from typing import List, Dict, Any

from google.genai import types
from google.adk.runners import InMemoryRunner, Runner
from google.adk.sessions import InMemorySessionService

# 👇 Adjust this import to match your repo
# Option A: if you have an App object
from pawconnect_ai.agent import app  
# Option B (if needed): from pawconnect_ai.agent import pawconnect_main_agent


2025-12-08 16:55:27.040 | INFO     | pawconnect_ai.agent:<module>:1225 - ADK root_agent created successfully with Vertex AI Gemini
2025-12-08 16:55:27.042 | INFO     | pawconnect_ai.agent:<module>:1226 - Project: gen-lang-client-0290817928, Region: us-central1
2025-12-08 16:55:27.042 | INFO     | pawconnect_ai.agent:<module>:1227 - Custom VertexAI Gemini class configured
2025-12-08 16:55:27.043 | INFO     | pawconnect_ai.agent:<module>:1228 - Function tools registered: search_pets, get_rescue_contact, schedule_visit
2025-12-08 16:55:27.043 | INFO     | pawconnect_ai.agent:<module>:1255 - ADK App created successfully


In [3]:
session_service = InMemorySessionService()

# If you expose `app`
runner: Runner = InMemoryRunner(app=app)

# If instead you have a root Agent called `pawconnect_main_agent`,
# use this alternative:
# from pawconnect_ai.agent import pawconnect_main_agent
# runner: Runner = InMemoryRunner(agent=pawconnect_main_agent)

USER_ID = "debug-user"
SESSION_ID = "pawconnect_debug_session"
APP_NAME = runner.app_name  # filled by ADK

App name mismatch detected. The runner is configured with app name "pawconnect_ai", but the root agent was loaded from "C:\Users\lwhieldon\KaggleAgentsCapstone\PawConnect\venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


In [4]:
from google.adk.events import Event

def summarize_event(event: Event) -> Dict[str, Any]:
    """Turn a raw ADK Event into a compact dict for visualization."""
    text = None
    if event.content and event.content.parts:
        # The first text part is usually what you care about in logs
        if getattr(event.content.parts[0], "text", None):
            text = event.content.parts[0].text

    calls = event.get_function_calls()
    responses = event.get_function_responses()

    if calls:
        kind = "tool_call"
    elif responses:
        kind = "tool_result"
    elif event.actions and (event.actions.state_delta or event.actions.artifact_delta):
        kind = "state_or_artifact"
    else:
        kind = "message"

    return {
        "id": event.id,
        "invocation_id": event.invocation_id,
        "author": event.author,              # 'user', 'pawconnect_main_agent', 'PetSearchAgent', etc.
        "branch": getattr(event, "branch", None),
        "kind": kind,
        "text": text,
        "function_calls": [
            {"name": c.name, "args": c.args}
            for c in (calls or [])
        ],
        "function_responses": [
            {"name": r.name, "response": r.response}
            for r in (responses or [])
        ],
        "state_delta": event.actions.state_delta if event.actions else None,
        "artifact_delta": event.actions.artifact_delta if event.actions else None,
        "timestamp": event.timestamp,
    }


In [5]:
async def run_and_trace(prompt: str) -> List[Dict[str, Any]]:
    # 1) Create a fresh session for this run
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    print(f"### Created new session: {session.id}\n")

    events_summary: List[Dict[str, Any]] = []

    # 2) Stream events
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=session.id,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=prompt)],
        ),
    ):
        summary = summarize_event(event)
        events_summary.append(summary)

        # Nicely formatted debug output (similar to the course notebooks)
        author = summary["author"]
        kind = summary["kind"]

        if kind == "tool_call":
            for call in summary["function_calls"]:
                print(
                    f"{author} > [Calling tool: {call['name']}({call['args']})]"
                )
        elif kind == "tool_result":
            for resp in summary["function_responses"]:
                print(
                    f"{author} > [Tool result: {resp['name']} -> {resp['response']}]"
                )
        elif summary["text"]:
            print(f"{author} > {summary['text']}")
        else:
            # State / artifacts / control signals
            print(f"{author} > ({kind}) {summary['state_delta'] or summary['artifact_delta']}")

    # 3) Also show the stored history from the session (every event)
    session = await runner.session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )
    print("\n\n### Session events (raw text stream)\n")
    for e in session.events:
        if e.content and e.content.parts and getattr(e.content.parts[0], "text", None):
            print(f"{e.content.role}: {e.content.parts[0].text}")

    return events_summary

In [6]:
events = await run_and_trace(
    "I live in Seattle and want to adopt a medium-sized dog that's good with cats and children, suitable for apartment living, and low-maintenance (first-time owner)."
)

### Created new session: pawconnect_debug_session

pawconnect_ai > Okay! I can help you find a medium-sized dog in the Seattle area. Since this is your first time owning a dog and you're living in an apartment, I'll focus on breeds known to be good with cats and children, and that are relatively low-maintenance.

Do you have a specific zip code I should use for the search?



### Session events (raw text stream)

user: I live in Seattle and want to adopt a medium-sized dog that's good with cats and children, suitable for apartment living, and low-maintenance (first-time owner).
model: Okay! I can help you find a medium-sized dog in the Seattle area. Since this is your first time owning a dog and you're living in an apartment, I'll focus on breeds known to be good with cats and children, and that are relatively low-maintenance.

Do you have a specific zip code I should use for the search?



In [7]:
import pandas as pd

df = pd.DataFrame(events)[
    [
        "timestamp",
        "author",
        "branch",
        "kind",
        "text",
        "function_calls",
        "function_responses",
    ]
].sort_values("timestamp")

df


,timestamp,author,branch,kind,text,function_calls,function_responses
0,1.765231e+09,pawconnect_ai,None,message,Okay! I can help you find a medium-sized dog i...,[],[]


In [8]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(app=app)

response = await runner.run_debug(
    "Help me match a senior dog with a quiet foster home.",
    verbose=True,
)


App name mismatch detected. The runner is configured with app name "pawconnect_ai", but the root agent was loaded from "C:\Users\lwhieldon\KaggleAgentsCapstone\PawConnect\venv\Lib\site-packages\google\adk\agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Help me match a senior dog with a quiet foster home.
pawconnect_ai > Okay! Do you have a location you would like me to search in?

